In [9]:
!pip install jieba

  Created wheel for jieba: filename=jieba-0.42.1-cp37-none-any.whl size=19314482 sha256=56224a34d550fdaf9e3cbe8ab4cc688eb2ebc0828d1c5d8c8d949fa96f142d16
  Stored in directory: C:\Users\WIN10\AppData\Local\pip\Cache\wheels\af\e4\8e\5fdd61a6b45032936b8f9ae2044ab33e61577950ce8e0dec29
Successfully built jieba


In [1]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, losses, metrics
from tensorflow.keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import jieba
import re
#from konlpy.tag import Okt

<br>
<br>
# 데이터 로드

Seq2Seq에서의 임베딩이 이전 예제와 다른 점은 아래와 같이 태그를 사용한다는 것입니다. 임베딩의 0~3번째에 각각 PADDING, START, END, OOV 태그를 넣습니다. 사실 그냥 똑같은 단어라고 보시면 됩니다. 다만 이 단어들이 Seq2Seq의 동작을 제어합니다. 

예를 들어, 디코더 입력에 START가 들어가면 디코딩의 시작을 의미합니다. 반대로 디코더 출력에 END가 나오면 디코딩을 종료합니다.
<br>
<br>
<br>

In [2]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 30

# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 128

# 정규 표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

# 챗봇 데이터 로드
csv_path = r"C:\Users\WIN10\Desktop\UCCProject\ChineseChatBotProject\chaotbotproject\chatbot\model\Total_dataset.xlsx"

chatbot_data = pd.read_excel(csv_path)
question, answer = list(chatbot_data['Q']), list(chatbot_data['A'])

챗봇의 훈련을 위해서 송영숙님이 공개한 한글 데이터셋을 로드합니다. 질문과 대답, 감정 등 총 3개의 항목으로 구성되어 있습니다. 감정 분류는 Seq2Seq에 필요가 없기 때문에 사용하지 않습니다.

https://github.com/songys/Chatbot_data
<br>
<br>
<br>

In [3]:
# 데이터 개수
len(question)


899

In [4]:
# 데이터의 일부만 학습에 사용
# question = question[:100]
# answer = answer[:100]

# 챗봇 데이터 출력
for i in range(10):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


Q : 12点  错！
A : 一天 又 过去 了

Q : 第一 志愿 学校 落榜 了
A : 慰劳 您

Q : 想 去 玩 4天3夜
A : 旅行 什么时候 都 好

Q : 想 出去 玩 四天三夜
A : 旅行 什么时候 都 好

Q : PPL 太 过分 了
A : 皱起 眉头 吧

Q : SD卡 坏 了
A : 重新 买 新 更 放心 了

Q : SD卡 不行
A : 重新 买 新 更 放心 了

Q : 为什么 不 玩 SNS 双臂 呢    
A : 也有 可能 不知道

Q : 我 知道 SNS 浪费时间   但是 每天 都 在 做
A : 定下 时间 再 做 吧

Q : SNS 是 浪费 时间 ， 但是 总 想 看
A : 定下 时间 再 做 吧



<br>
<br>
# 단어 사전 생성

In [5]:
# 형태소분석 함수
def pos_tag(sentences):
    #중국어 형태소 분석기 설정
    #j.enable_paddle()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        # 특수기호 제거
        sentence = re.sub(RE_FILTER, "", sentence)
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        #sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        print(sentences)
        
    return sentences_pos

문장을 먼저 최소 단위인 토큰으로 나누어야 합니다. 한글 문장이라면 보통 KoNLPy(코엔엘파이)를 통해 형태소분석을 적용합니다. Hannanum, Kkma, Komoran, Mecab, Okt(Twitter) 등 여러가지 라이브러리를 동일한 인터페이스로 실행이 가능합니다.

KoNLPy는 자바로 구현이 되어 있습니다. 맥OS와 달리 윈도우 아나콘다는 아래와 같은 작업을 거쳐야 합니다.

< 1. 자바 설치 ><br>
커맨드창에서 'java -version' 명령어를 실행하여 버전이 1.7 이상이어야 합니다.

< 2. JPype1 설치 ><br>
파이썬에서 자바를 호출할 수 있는 라이브러리입니다.

< 3. KoNLPy 설치 ><br>
위의 두 가지가 미리 완료되어 있어야만 가능합니다.

아래 링크에서 좀 더 자세한 방법을 확인하시기 바랍니다.<br>
-> https://ericnjennifer.github.io/python_visualization/2018/01/21/PythonVisualization_Chapt1.html
<br>
<br>
<br>

In [6]:
# 형태소분석 수행
question = pos_tag(question)
answer = pos_tag(answer)

# 형태소분석으로 변환된 챗봇 데이터 출력
for i in range(10):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


 轻快 的 音乐 。', '为什么呢 ？', '想一想 是 什么 原因 。', '比起 一个人 住 ， 应该 不太 方便 吧 。', '下学期 要 更加 努力 地 管理学 分。', '比起 一个人 住 ， 应该 不太 方便 吧 。', '比起 一个人 住 ， 应该 不太 方便 吧 。', '下学期 要 更加 努力 地 管理学 分。', '多 了解 技术 会 有 帮助 。', '是 梦寐以求 的 旅行 。', '是 梦寐以求 的 旅行 。', '真是 郁闷 的 情况 。', '真是 郁闷 的 情况 。', '那 该 多好 啊 ！', '直接 给 好像 更好 。', '直接 给 好像 更好 。', '直接 给 好像 更好 。', '应该 挺好 的 吧 。', '直接 给 好像 更好 。', '会 有 更好 的 机会 的 。', '会 有 更好 的 机会 的 。', '准备 艺人 ， 应该 比 一般人 漂亮 吧 。', '准备 艺人 ， 应该 比 一般人 漂亮 吧 。', '所以 我 不能 留 长 就 剪 。', '所以 我 不能 留 长 就 剪 。', '幸好 现在 好多 了 。', '请 您 大喘 口气', '请 您 大喘 口气', '不要 提前 紧张 。', '喜欢 的 话 就 给 我 吧 。', '我 也 不 喜欢 。', '好好 干 吧。', '喜欢 的 话 就 给 我 吧 。', '好好 干 吧。', '即使 这样 ， 我 也 能 超越 。', '小心 点 。', '别 太 灰心。', '小心 点 。', '太 滑 了 ， 小心 点 。', '为了 健康 一点一点 吃 吧 。', '去 超市 吧 。', '祝 你 用餐 愉快 。', '祝 你 用餐 愉快 。', '好吃 吧 ！', '好像 等 了 很久 。', '再 等 一会儿 ， 就会 重新 通电 的 。', '适当 。', '请 自己 变得 坚强 吧 。', '我 想 给 你 。', '家里 的 气氛 会 改变 的 。', '家里 的 气氛 会 改变 的 。', '送花 什么 时候 都是 好 的 。', '送花 什么 时候 都是 好 的 。', '手艺 真好 。', '看起来 心情 很好 。', '看起来 心情 很好 。', '是 保持 心灵 安定 的 好 爱好 。', '是 保持 心灵 安定 的 好 爱

In [8]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))
print(len(words))
# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]
print(words)

1722
['<PADDING>', '<START>', '<END>', '<OOV>', '自责', '骂人', '过来', '一般', '机关', '之', '察觉', '猎杀', '离家出走', '学习', '一旦', '愉快', '慌', '第一位', '交', '镜子', '赢', '只能', '决定', '时间', '遗漏', '有点', '有种', '如', '国家', '谴责', '下学期', '坏话', '明显', '疏忽', '鲁莽', '无关紧要', '资格证', '简单', '戒烟', '很可爱', '别', '艺人', '偷偷地', '当然', '期待', '人缘', '回家', '在家里', '明天', '给以', '开学', '改写', '眉头', '整理', '剪', '午觉', '纤维', '虽然', '夜班', '放任', '乳液', '感谢', '，干嘛', '…', '教职', '治疗', '嚼', '带来', '想起', '渴', '捐', '太帅', '开玩笑', '能力', '待遇', '浪费', '越好', '放', '开讲', '喜事', '问一下', '老师', '凉水', '开发', '洗', '饮食，坚持运动吧。', '动', '短', '创业', '想见', '怪物', '决断', '出去', '不一样', '懒惰', '出现', '不够', '带', '山', '为了', '坚果类', '院', '勉强', '喜悦', '感冒', '看来', '什么', '看起来', '那么', '迫切', '看着', '心痛', '想一想', '偶尔', '对话', '同班', '推荐', '忘记', '真正', '地方', '轻快', '方法', '队伍', '解决好', '就是', '放弃', '出', '帮忙', '蜂拥而去', '概念', '还没有', '于', '经验', '掉', '足够', '自尊感', '路上', '他', '快点', '关系', '头脑', '打赌', '机会', '何时', '证据', '送走', '迈出', '那时', '好可惜', '观念', '教养', '肥肠', '傻。', '早知道', '意见', '以', '没肺腑', '错过了', '加', '出汗', '过', '多

질문과 대답 문장들을 합쳐서 전체 단어 사전을 만듭니다. 자연어처리에서는 항상 이렇게 단어를 인덱스에 따라 정리를 해야 합니다. 그래야지 문장을 인덱스 배열로 바꿔서 임베딩 레이어에 넣을 수 있습니다. 또한 모델의 출력에서 나온 인덱스를 다시 단어로 변환하는데도 필요합니다.
<br>
<br>
<br>

In [10]:
# 단어 개수
len(words)


1726

In [11]:
# 단어 출력
words[:20]


['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '就要',
 '一般人',
 '空虚',
 '家庭关系',
 '坏话',
 '道歉',
 '前面',
 '人际',
 '谎话',
 '钓鱼',
 '炒鱿鱼',
 '火车',
 '渴',
 '单方面',
 '院',
 '墙']

In [12]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

print(index_to_word)

{0: '<PADDING>', 1: '<START>', 2: '<END>', 3: '<OOV>', 4: '就要', 5: '一般人', 6: '空虚', 7: '家庭关系', 8: '坏话', 9: '道歉', 10: '前面', 11: '人际', 12: '谎话', 13: '钓鱼', 14: '炒鱿鱼', 15: '火车', 16: '渴', 17: '单方面', 18: '院', 19: '墙', 20: '家人们', 21: '气', 22: '呗', 23: '鼓励', 24: '天天', 25: '大小', 26: '无法', 27: '很可爱', 28: '法', 29: '宾客', 30: '睡', 31: '笨', 32: '同', 33: '有趣', 34: '改掉', 35: '初次', 36: '每当', 37: '可以', 38: '没关系', 39: '发痒', 40: '一开始', 41: '嗯', 42: '祝福', 43: '会好', 44: '家务。', 45: '是否', 46: '确定', 47: '话', 48: '，哥', 49: '对不对', 50: '属于', 51: '炸鸡', 52: '把', 53: '好吃', 54: '无语', 55: '那', 56: '被套', 57: '笑笑', 58: '空腹', 59: '不错', 60: '久', 61: '正确', 62: '主意', 63: '实现', 64: '闹着玩', 65: '完全', 66: '车祸', 67: '站', 68: '运动', 69: '幸', 70: '公事', 71: '发软。', 72: '言', 73: '手机', 74: '很漂亮', 75: '洗', 76: '变得', 77: '家人', 78: '推荐', 79: '自己', 80: '精神失常', 81: '告白', 82: '怀着', 83: '谎', 84: '更', 85: '行李', 86: '剩下', 87: '搞', 88: '如同', 89: '兜兜风。', 90: '胶鞋', 91: '重重', 92: '跟上', 93: '谢谢', 94: '很紧张', 95: '而', 96: '差异', 97: '慢慢', 98: '关机', 99: 

In [13]:
# 단어 -> 인덱스
# 문장을 인덱스로 변환하여 모델 입력으로 사용
dict(list(word_to_index.items())[:20])


{'<PADDING>': 0,
 '<START>': 1,
 '<END>': 2,
 '<OOV>': 3,
 '就要': 4,
 '一般人': 5,
 '空虚': 6,
 '家庭关系': 7,
 '坏话': 8,
 '道歉': 9,
 '前面': 10,
 '人际': 11,
 '谎话': 12,
 '钓鱼': 13,
 '炒鱿鱼': 14,
 '火车': 15,
 '渴': 16,
 '单方面': 17,
 '院': 18,
 '墙': 19}

In [14]:
# 인덱스 -> 단어
# 모델의 예측 결과인 인덱스를 문장으로 변환시 사용
dict(list(index_to_word.items())[:20])


{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: '就要',
 5: '一般人',
 6: '空虚',
 7: '家庭关系',
 8: '坏话',
 9: '道歉',
 10: '前面',
 11: '人际',
 12: '谎话',
 13: '钓鱼',
 14: '炒鱿鱼',
 15: '火车',
 16: '渴',
 17: '单方面',
 18: '院',
 19: '墙'}

<br>
<br>
# 전처리

In [15]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)
        print(sentences_index)
    return np.asarray(sentences_index)

Seq2Seq에서는 학습시 다음과 같이 총 3개의 데이터가 필요합니다.

인코더 입력 : 12시 땡<br>
디코더 입력 : START 하루 가 또 가네요<br>
디코더 출력 : 하루 가 또 가네요 END

원래 Seq2Seq는 디코더의 현재 출력이 디코더의 다음 입력으로 들어갑니다. 다만 학습에서는 굳이 이렇게 하지 않고 디코더 입력과 디코더 출력의 데이터를 각각 만듭니다. 

그러나 예측시에는 이런 방식이 불가능합니다. 출력값을 미리 알지 못하기 때문에, 디코더 입력을 사전에 생성할 수가 없습니다. 이런 문제를 해결하기 위해 훈련 모델과 예측 모델을 따로 구성해야 합니다. 모델 생성 부분에서 다시 자세히 설명을 드리겠습니다.
<br>
<br>
<br>

In [16]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)

# 첫 번째 인코더 입력 출력 (12시 땡)
x_encoder[0]


, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [109, 682, 113, 1683, 165, 1202, 707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [109, 560, 854, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [109, 1576, 1645, 1537, 451, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [109, 1180, 1446, 970, 1627, 1342, 83, 520, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [109, 613, 1158, 964, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [109, 1612, 1602, 1104, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [109, 1612, 430, 1021, 455, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [109, 1612, 1491, 1627, 707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [109, 65, 245, 1627, 107, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [10

array([306, 853,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [17]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

# 첫 번째 디코더 입력 출력 (START 하루 가 또 가네요)
x_decoder[0]


 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1346, 304, 1102, 707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 109, 851, 683, 707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 365, 79, 56, 558, 1627, 573, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 807, 960, 707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1626, 107, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 671, 1698, 1568, 639, 155, 707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1631, 910, 148, 944, 155, 707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 721, 613, 1564, 964, 707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 960, 504, 602, 707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1204, 1052, 1627, 707, 0, 0, 0, 0, 0, 

array([   1,  922,  740,  869, 1627,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [18]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)

# 첫 번째 디코더 목표 출력 (하루 가 또 가네요 END)
y_decoder[0]


 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1346, 304, 1102, 707, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [109, 851, 683, 707, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [365, 79, 56, 558, 1627, 573, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [807, 960, 707, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1626, 107, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [671, 1698, 1568, 639, 155, 707, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1631, 910, 148, 944, 155, 707, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [721, 613, 1564, 964, 707, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [960, 504, 602, 707, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1204, 1052, 1627, 707, 2, 0, 0, 0, 0, 0, 

array([ 922,  740,  869, 1627,    2,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [19]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)))

# 디코더 목표를 원핫인코딩으로 변환
# 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
        one_hot_data[i, j, index] = 1

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫 번째 디코더 목표 출력
y_decoder[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

인코더 입력과 디코더 입력은 임베딩 레이어에 들어가는 인덱스 배열입니다. 반면에 디코더 출력은 원핫인코딩 형식이 되어야 합니다. 디코더의 마지막 Dense 레이어에서 softmax로 나오기 때문입니다.

<br>
<br>
# 모델 생성

In [20]:
#--------------------------------------------
# 훈련 모델 인코더 정의
#--------------------------------------------

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                dropout=0.2,
                                                recurrent_dropout=0.4,
                                                return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]



#--------------------------------------------
# 훈련 모델 디코더 정의
#--------------------------------------------

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           dropout=0.1,
                           recurrent_dropout=0.5,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)



#--------------------------------------------
# 훈련 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])    

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


지금까지의 예제는 Sequential 방식의 모델이었습니다. 하지만 이번에는 함수형 API 모델을 사용했습니다. 인코더와 디코더가 따로 분리되어야 하는데, 단순히 레이어를 추가하여 붙이는 순차형으로는 구현이 불가능하기 때문입니다. 

Model() 함수로 입력과 출력을 따로 설정하여 모델을 만듭니다. 그다음 compile과 fit은 이전과 동일하게 적용하시면 됩니다.
<br>
<br>
<br>

In [21]:
#--------------------------------------------
#  예측 모델 인코더 정의
#--------------------------------------------

# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)



#--------------------------------------------
# 예측 모델 디코더 정의
#--------------------------------------------

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

예측 모델은 이미 학습된 훈련 모델의 레이어들을 그대로 재사용합니다. 예측 모델 인코더는 훈련 모델 인코더과 동일합니다. 그러나 예측 모델 디코더는 매번 LSTM 상태값을 입력으로 받습니다. 또한 디코더의 LSTM 상태를 출력값과 같이 내보내서, 다음 번 입력에 넣습니다. 

이렇게 하는 이유는 LSTM을 딱 한번의 타임 스텝만 실행하기 때문입니다. 그래서 매번 상태값을 새로 초기화 해야 합니다. 이와 반대로 훈련할때는 문장 전체를 계속 LSTM으로 돌리기 때문에 자동으로 상태값이 전달됩니다. 

<br>
<br>
# 훈련 및 테스트

In [22]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        if vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else: 
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence.extend([vocabulary[OOV_INDEX]])
            
        # 빈칸 추가
        sentence += ' '

    return sentence

In [23]:
# 에폭 반복
for epoch in range(5):
    print('Total Epoch :', epoch + 1)

    # 훈련 시작
    history = model.fit([x_encoder, x_decoder],
                        y_decoder,
                        epochs=100,
                        batch_size=64,
                        verbose=0)
    
    # 정확도와 손실 출력
    print('accuracy :', history.history['acc'][-1])
    print('loss :', history.history['loss'][-1])
    
    # 문장 예측 테스트
    # (3 박 4일 놀러 가고 싶다) -> (여행 은 언제나 좋죠)
    input_encoder = x_encoder[5].reshape(1, x_encoder[2].shape[0])
    input_decoder = x_decoder[5].reshape(1, x_decoder[2].shape[0])
    print(input_encoder)
    results = model.predict([input_encoder, input_decoder])
    
    # 결과의 원핫인코딩 형식을 인덱스로 변환
    # 1축을 기준으로 가장 높은 값의 위치를 구함
    indexs = np.argmax(results[0], 1) 
    
    # 인덱스를 문장으로 변환 
    sentence = convert_index_to_text(indexs, index_to_word)
    print(sentence)
    print()

encoder_model.save("encoder_model3.h5")
decoder_model.save("decoder_model3.h5")    

Total Epoch : 1
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
accuracy : 0.9049685
loss : 0.45593597725845947
[[1040  760 1627    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]]
请 太 确信 更 放心 了 

Total Epoch : 2
accuracy : 0.95394886
loss : 0.2114381655735752
[[1040  760 1627    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]]
请 买 新 更 放心 了 

Total Epoch : 3
accuracy : 0.9610308
loss : 0.15622646257994038
[[1040  760 1627    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]]
去 买 新 更 放心 了 

Total Epoch : 4
accuracy : 0.9661847
loss : 0.12793525140131673
[[1040  760 1627    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0  

학습이 진행될수록 예측 문장이 제대로 생성되는 것을 볼 수 있습니다. 다만 여기서의 예측은 단순히 테스트를 위한 것이라, 인코더 입력과 디코더 입력 데이터가 동시에 사용됩니다. 아래 문장 생성에서는 예측 모델을 적용하기 때문에, 오직 인코더 입력 데이터만 집어 넣습니다.

<br>
<br>
# 문장 생성

In [25]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [30]:
# 텍스트 생성
from tensorflow.keras.models import load_model

def generate_text(input_seq):

    encoder_model = load_model("encoder_model3.h5")    
    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)

    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX
    
    # 인덱스 초기화
    indexs = []
    
    decoder_model = load_model("decoder_model3.h5")
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)
        
        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index
        
        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)    
    return sentence

제일 첫 단어는 START로 시작합니다. 그리고 출력으로 나온 인덱스를 디코더 입력으로 넣고 다시 예측을 반복합니다. 상태값을 받아 다시 입력으로 같이 넣는 것에 주의하시기 바랍니다. END 태그가 나오면 문장 생성을 종료합니다.
<br>
<br>
<br>

In [31]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('我 爱 你')
input_seq


['我 爱 你']
[[109, 1493, 960, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


array([[ 109, 1493,  960,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])

In [32]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


'找找 你 真正 想 做 的 事情 吧 。 '

데이터셋에 있는 문장과 똑같은 입력을 넣으니, 역시 정확히 일치하는 답변이 출력되었습니다.
<br>
<br>
<br>

In [ ]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('3박4일 같이 놀러가고 싶다')
input_seq


array([[171,   3,   3,   3,   3,   3,   3,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [ ]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


'全家 都 去 称 心 如意 的 地方 吧 '

데이터셋 문장에서는 없던 '같이'라는 단어를 추가해 보았습니다. 그래도 비슷한 의미란 것을 파악하여 동일한 답변이 나왔습니다.
<br>
<br>
<br>

In [ ]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('3박4일 놀러가려고')
input_seq


array([[171,   3,   3,   3,   3,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [ ]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


'全家 都 去 称 心 如意 的 地方 吧 '

하지만 데이터셋에 없던 '가려고'로 입력을 수정하니, 전혀 다른 문장이 출력되었습니다. 이는 우리가 데이터의 일부인 100개 문장만 학습했기 때문입니다. 데이터의 개수를 늘려서 훈련할수록 일반화 능력이 더욱 높아집니다.